
# 2. Données

Le jeu de données peut être téléchargé depuis : [https://www.kaggle.com/ntnu-testimon/paysim1](https://www.kaggle.com/ntnu-testimon/paysim1)

Ce jeu de données contient des transactions d'argent mobile qui ont été générées avec le simulateur PaySim. La simulation était basée sur un échantillon de transactions réelles collectées par une entreprise qui est le fournisseur du service financier mobile actuellement en cours dans plus de 14 pays à travers le monde. Les données sont un ensemble de journaux financiers d'un mois d'un service d'argent mobile mis en place dans un pays africain.

Les données sont regroupées dans un fichier CSV. Un exemple de ligne d'échantillon :
\`1,PAYMENT,1060.31,C429214117,1089.0,28.69,M1591654462,0.0,0.0,0,0\`

Le jeu de données contient (suivant l'exemple ci-dessus) :

- **step** - représente une unité de temps dans le monde réel. Dans ce cas, 1 étape équivaut à 1 heure de temps. Total des étapes : 744 (simulation de 30 jours).
- **type** - CASH-IN, CASH-OUT, DEBIT, PAYMENT et TRANSFER.
- **amount** - montant de la transaction en monnaie locale.
- **nameOrig** - client ayant initié la transaction.
- **oldbalanceOrg** - solde initial avant la transaction.
- **newbalanceOrig** - nouveau solde après la transaction.
- **nameDest** - client destinataire de la transaction.
- **oldbalanceDest** - solde initial du destinataire avant la transaction. Remarque : aucune information n'est disponible pour les clients dont le nom commence par M (commerçants).
- **newbalanceDest** - nouveau solde du destinataire après la transaction. Remarque : aucune information n'est disponible pour les clients dont le nom commence par M (commerçants).
- **isFraud** - Il s'agit des transactions effectuées par des agents frauduleux dans la simulation. Dans ce jeu de données spécifique, le comportement frauduleux des agents vise à tirer profit en prenant le contrôle des comptes des clients et en essayant de vider les fonds en les transférant vers un autre compte, puis en les retirant du système.
- **isFlaggedFraud** - Le modèle commercial vise à contrôler les transferts massifs d'un compte à un autre et signale les tentatives illégales. Une tentative illégale dans ce jeu de données est une tentative de transférer plus de 200 000 lors d'une seule transaction.


In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb

# Model evaluations
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Models from Scikit-Learn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

: 

# Load and Exploration Data

In [ ]:
df = pd.read_csv("data.csv")
df.info()

In [ ]:
df.head()

In [ ]:
dt = df.copy()

In [ ]:
dt.type.unique()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
data = df.copy()

In [ ]:
#Nombre total de transactions
total_transactions = len(data)


In [ ]:
# Montant total des transactions
total_amount = data['amount'].sum()

In [ ]:
# Fréquence des types de transactions
transaction_types_frequency = data['type'].value_counts()

In [ ]:

#  Fréquence des types de transactions avec visualisation graphique
transaction_types_frequency = data['type'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=transaction_types_frequency.index, y=transaction_types_frequency.values)
plt.title('Fréquence des types de transactions')
plt.xlabel('Type de transaction')
plt.ylabel('Nombre de transactions')
plt.show()

In [ ]:
# Analyse du solde avant et après la transaction
data['balance_change'] = data['newbalanceOrig'] - data['oldbalanceOrg']

In [ ]:
#Répartition des soldes avant et après les transactions avec visualisation graphique
plt.figure(figsize=(12, 6))
sns.kdeplot(data['oldbalanceOrg'], label='Solde avant')
sns.kdeplot(data['newbalanceOrig'], label='Nouveau solde')
plt.title('Répartition des soldes avant et après les transactions')
plt.xlabel('Solde')
plt.ylabel('Densité')
plt.legend()
plt.show()

In [ ]:
#  Nombre de transactions frauduleuses
fraudulent_transactions = data[data['isFraud'] == 1]
num_fraudulent_transactions = len(fraudulent_transactions)

In [ ]:
# Pourcentage de transactions frauduleuses
percentage_fraudulent_transactions = (num_fraudulent_transactions / total_transactions) * 100


In [ ]:
# Fréquence des transferts illégaux signalés
flagged_fraud_transactions = data[data['isFlaggedFraud'] == 1]
num_flagged_fraud_transactions = len(flagged_fraud_transactions)
num_flagged_fraud_transactions 


In [ ]:
# enregistrer les transaction frauduleuse 
flagged_fraud_transactions.to_csv('transaction_frauduleuse.csv')
transaction_frauduleuse = pd.read_csv('transaction_frauduleuse.csv')
transaction_frauduleuse

In [ ]:
# Montant moyen des transactions par type
average_amount_by_type = data.groupby('type')['amount'].mean()

In [ ]:

# Analyse de la fraude par type de transaction
fraud_by_type = fraudulent_transactions['type'].value_counts()

In [ ]:

# Taux de fraude par unité de temps
fraud_rate_by_step = fraudulent_transactions['step'].value_counts() / total_transactions



In [ ]:

# Analyse des transactions de commerçants
merchant_transactions = data[data['nameDest'].str.startswith('M')]
num_merchant_transactions = len(merchant_transactions)


 le pourcentage de transactions frauduleuses qui ont été correctement signalées (isFlaggedFraud).

In [ ]:
correctly_flagged_rate = len(flagged_fraud_transactions) / num_fraudulent_transactions
print(" Taux de transactions frauduleuses signalées correctement =", correctly_flagged_rate)


##### Analyse du comportement des clients frauduleuse 

In [ ]:
fraudulent_clients_stats = fraudulent_transactions.groupby('nameOrig').agg(
    count_amount=('amount', 'count'),
    mean_amount=('amount', 'mean'),
    mean_balance_change=('balance_change', 'mean')
)
fraudulent_clients_stats['count_amount'].unique()

In [ ]:
# Enregistrez les résultats dans un fichier CSV
fraudulent_clients_stats.to_csv('Comportement_desclient_frauduleuse.csv')
compor_client = pd.read_csv('Comportement_desclient_frauduleuse.csv')
compor_client.info()

In [ ]:
fraudulent_clients_stats

L'analyse que l'on peut faire à partir des résultats montre que, en moyenne, les clients frauduleux transfèrent ou retirent des montants équivalents à leur solde, et chacun d'entre eux effectue une seule transaction.

 le temps moyen écoulé entre la création d'une transaction frauduleuse et sa détection.

In [ ]:
fraud_detection_time = (fraudulent_transactions['step'] - fraudulent_transactions['step'].min()).mean()
print("Temps moyen de détection des fraudes =", fraud_detection_time)


 la moyenne des montants pour les transactions marquées comme frauduleuses.

In [ ]:
average_fraudulent_amount = fraudulent_transactions['amount'].mean()
print(" Montant moyen de transactions frauduleuses =", average_fraudulent_amount)


 les heures de la journée où les transactions sont les plus fréquentes.

In [ ]:
peak_hours = data.groupby('step')['step'].count().idxmax()
print(" Heures de pointe =", peak_hours)


la fréquence à laquelle les soldes des comptes sont renouvelés.

In [ ]:
balance_renewal_rate = (data['newbalanceOrig'] > data['oldbalanceOrg']).sum() / total_transactions
print("Taux de renouvellement des soldes =", balance_renewal_rate)


les catégories de destinataires les plus courantes dans les transactions.

In [ ]:
top_destination_categories = data['nameDest'].apply(lambda x: x[0] if x[0] != 'M' else 'Marchant').value_counts()
print("Catégories de destinataires les plus fréquentes :\n", top_destination_categories)


In [ ]:
# Analyse des catégories de destinataires les plus fréquentes avec visualisation graphique
top_destination_categories = data['nameDest'].apply(lambda x: x[0] if x[0] != 'M' else 'Marchant').value_counts()
plt.figure(figsize=(10, 6))
top_destination_categories.plot(kind='pie', autopct='%1.1f%%')
plt.title('Répartition des catégories de destinataires')
plt.show()

In [ ]:
average_transaction_duration = data.groupby('type')['step'].diff().mean()
print(" Durée moyenne des transactions =", average_transaction_duration)


In [ ]:
#  Évolution temporelle des transactions avec visualisation graphique
transactions_by_time = data.groupby('step')['step'].count()
plt.figure(figsize=(14, 6))
transactions_by_time.plot(kind='line')
plt.title('Évolution temporelle des transactions')
plt.xlabel('Heure (step)')
plt.ylabel('Nombre de transactions')
plt.show()

#### etude sur le variable montant 

In [ ]:
#  Répartition des montants de transactions par type avec visualisation graphique
plt.figure(figsize=(12, 6))
sns.boxplot(x='type', y='amount', data=data)
plt.title('Répartition des montants de transactions par type')
plt.xlabel('Type de transaction')
plt.ylabel('Montant de la transaction')
plt.show()


In [ ]:
# Taux de fraude par catégorie de destinataire avec visualisation graphique
data['dest_category'] = data['nameDest'].apply(lambda x: x[0] if x[0] != 'M' else 'Marchant')
fraud_rate_by_dest_category = data.groupby('dest_category')['isFraud'].mean()
plt.figure(figsize=(10, 6))
fraud_rate_by_dest_category.plot(kind='bar')
plt.title('Taux de fraude par catégorie de destinataire')
plt.xlabel('Catégorie de destinataire')
plt.ylabel('Taux de fraude')
plt.show()


In [ ]:
#  Répartition des montants de transactions par heure avec visualisation graphique
plt.figure(figsize=(14, 8))
sns.boxplot(x='step', y='amount', data=data)
plt.title('Répartition des montants de transactions par heure')
plt.xlabel('Heure (step)')
plt.ylabel('Montant de la transaction')
plt.show()


In [ ]:
#  Diagramme en secteurs des types de transactions frauduleuses 
fraud_by_type = fraudulent_transactions['type'].value_counts()
plt.figure(figsize=(10, 6))
fraud_by_type.plot(kind='pie', autopct='%1.1f%%')
plt.title('Répartition des types de transactions frauduleuses')
plt.show()


In [ ]:

date_debut = pd.to_datetime('2023-02-01')  # la date de debut 

# Convertir step en heures en ajoutant la date de début
data['timestamp'] = date_debut + pd.to_timedelta(data['step'], unit='h')

# Extraire le jour de la semaine et ajouter une colonne 'jour_de_la_semaine'
data['jour_de_la_semaine'] = data['timestamp'].dt.day_name()

# Visualisation du résultat
#print(data[['step', 'timestamp', 'jour_de_la_semaine']].head())

data['timestamp'].unique()

In [ ]:
# Enregistrez les résultats dans un fichier CSV
#data.to_csv('resultats_dask.csv')
#up_data = pd.read_csv('resultats_dask.csv')
#up_data.info()

In [ ]:
transaction_frauduleuse_update  = data[data['isFraud'] == 1]
transaction_frauduleuse_update.to_csv('transaction_frauduleuse_update.csv')
up_data = pd.read_csv('transaction_frauduleuse_update.csv')
up_data.info()

In [ ]:

# Évolution temporelle des soldes moyens avant et après les transactions 
plt.figure(figsize=(14, 8))
plt.plot(data['timestamp'], data['oldbalanceOrg'], label='Solde initial moyen', linestyle='-', color='b')
plt.plot(data['timestamp'], data['newbalanceOrig'], label='Nouveau solde moyen', linestyle='-', color='g')
plt.title('Évolution temporelle des soldes moyens avant et après les transactions')
plt.xlabel('Timestamp')
plt.ylabel('Solde moyen')
plt.legend()
plt.show()

In [ ]:
# Montant moyen de transactions par heure avec visualisation graphique
average_amount_by_hour = data.groupby(data['timestamp'].dt.hour)['amount'].mean()
plt.figure(figsize=(10, 6))
average_amount_by_hour.plot(kind='bar', color='skyblue')
plt.title('Montant moyen de transactions par heure')
plt.xlabel('Heure du jour')
plt.ylabel('Montant moyen de la transaction')
plt.show()


In [ ]:
# Taux de fraude par jour de la semaine avec tendance lissée
fraud_rate_by_weekday = data.groupby(data['timestamp'].dt.day_name())['isFraud'].mean()
plt.figure(figsize=(10, 6))
fraud_rate_by_weekday.plot(kind='line', marker='o', color='red')
plt.title('Taux de fraude par jour de la semaine avec tendance lissée')
plt.xlabel('Jour de la semaine')
plt.ylabel('Taux de fraude')
plt.show()


In [ ]:
# Analyse des variations horaires des types de transactions 
plt.figure(figsize=(14, 8))
sns.countplot(x=data['timestamp'].dt.hour, hue=data['type'], data=data)
plt.title('Répartition horaire des types de transactions')
plt.xlabel('Heure du jour')
plt.ylabel('Nombre de transactions')
plt.legend(title='Type de transaction')
plt.show()


 les heures de la journée où les transactions sont les plus fréquentes se situe entre 18h et 20h avec un pic en 19h. Autre constact les transaction débute entre 7h et 8h et se termine entre 0 et 1h du matin

In [ ]:
#  Évolution du solde destinataire après la transaction
plt.figure(figsize=(14, 8))
sns.lineplot(x='timestamp', y='newbalanceDest', hue='jour_de_la_semaine', data=data)
plt.title("Évolution du solde destinataire après la transaction par jour")
plt.xlabel("Timestamp")
plt.ylabel("Solde destinataire après la transaction")
plt.legend(title='Jour de la semaine')
plt.show()


In [ ]:
fraudulent_transactions = data[data['isFraud']==1]
fraudulent_transactions

In [ ]:
# Analyse des variations horaires des types de transactions 
plt.figure(figsize=(14, 8))
sns.countplot(x=fraudulent_transactions['timestamp'].dt.hour, hue=data['isFraud'], data=data)
plt.title('Répartition horaire des types de transactions')
plt.xlabel('Heure du jour')
plt.ylabel('Nombre de transactions')
plt.legend(title='Type de transaction')
plt.show()


In [ ]:
# Distribution des montants de transactions frauduleuses par jour de la semaine avec visualisation graphique
plt.figure(figsize=(14, 8))
sns.boxplot(x='jour_de_la_semaine', y='amount', data=fraudulent_transactions)
plt.title('Distribution des montants de transactions frauduleuses par jour de la semaine')
plt.xlabel('Jour de la semaine')
plt.ylabel('Montant de la transaction')
plt.show()


In [ ]:
# Analyse de la variation horaire des transactions frauduleuses 
fraudulent_hours = fraudulent_transactions['timestamp'].dt.hour
plt.figure(figsize=(10, 6))
sns.histplot(fraudulent_hours, bins=24, kde=True)
plt.title('Analyse de la variation horaire des transactions frauduleuses')
plt.xlabel('Heure du jour')
plt.ylabel('Nombre de transactions frauduleuses')
plt.show()


In [ ]:
#Montant moyen des transactions par type de transaction 
plt.figure(figsize=(12, 6))
sns.barplot(x='type', y='amount', data=data, hue='isFraud', ci=None)
plt.title('Montant moyen des transactions par type de transaction')
plt.xlabel('Type de transaction')
plt.ylabel('Montant moyen de la transaction')
plt.show()


In [ ]:
#  Histogramme des montants de transactions frauduleuses avec visualisation graphique
plt.figure(figsize=(10, 6))
sns.histplot(fraudulent_transactions['amount'], bins=30, kde=True)
plt.title('Distribution des montants de transactions frauduleuses')
plt.xlabel('Montant de la transaction')
plt.ylabel('Fréquence')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

#  'timestamp' est au format datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Calculer le taux de fraude par heure
fraud_rate_by_hour = data.groupby('timestamp')['isFraud'].mean()

# KPI : Évolution du taux de fraude par timestamp avec visualisation graphique
plt.figure(figsize=(14, 8))
plt.plot(fraud_rate_by_hour.index, fraud_rate_by_hour.values, marker='o', linestyle='-', color='r')
plt.title('Évolution du taux de fraude par timestamp')
plt.xlabel('Timestamp')
plt.ylabel('Taux de fraude')
plt.show()


In [ ]:
# Heatmap de la fréquence des types de transactions par jour de la semaine avec visualisation graphique
transaction_types_by_weekday = data.groupby(['jour_de_la_semaine', 'type']).size().unstack()
plt.figure(figsize=(12, 8))
sns.heatmap(transaction_types_by_weekday, cmap='Blues', annot=True, fmt='d')
plt.title('Fréquence des types de transactions par jour de la semaine')
plt.xlabel('Type de transaction')
plt.ylabel('Jour de la semaine')
plt.show()


In [ ]:
#Répartition des montants de transactions par timestamp avec visualisation graphique
plt.figure(figsize=(14, 8))
fig1 = plt.scatter(data['timestamp'], data['amount'])
#fig2 = px.scatter(data, x="timestamp", y="amount")
plt.title('Répartition des montants de transactions par timestamp')
plt.xlabel('Timestamp')
plt.ylabel('Montant de la transaction')
plt.show()


In [ ]:
#Répartition des montants de transactions par timestamp avec visualisation graphique
plt.figure(figsize=(14, 8))
fig1 = plt.scatter(fraudulent_transactions['timestamp'], fraudulent_transactions['amount'])
#fig2 = px.scatter(data, x="timestamp", y="amount")
plt.title('Répartition des montants de transactions frauduleuse par timestamp')
plt.xlabel('Timestamp')
plt.ylabel('Montant de la transaction')
plt.show()

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report")
profile

##  Exécution des différents KPI avec duckdb

In [ ]:
import duckdb
duckdb.query("SELECT nameDest, amount FROM df where oldbalanceDest == 0.0").to_df()

In [ ]:
#spécifier l'axe des x et l'axe des y
ax = df['isFraud'].value_counts().plot(
kind='pie', # Pour créer un graphique circulaire avec matplotlib,vous pouvez utiliser la fonction 'pie'
figsize=(8,4),# déterminer la taille du graphique
autopct = '%.2f%%', 
labels=df['isFraud'].value_counts().index, #Fournir les étiquettes pour chaque type de TRANACTION 
legend = True,
)

ax.set_title(
'Distributions des transactions fauduleuse et non frauduleuse', # Titre du graphique
loc ='center', # Positionner au centre
fontsize=12, # Dimension du texte
fontweight='bold' # type de caractere
)

plt.show()

In [ ]:
# les client qui font des transaction massive de compte en comme 

Client_trans_massive  = duckdb.query("select nameOrig , amount from df where isFlaggedFraud == 1").to_df()
Client_trans_massive

In [ ]:
Client_trans_massive.max()

In [ ]:
rentabilité_transaction = duckdb.sql("""SELECT 
    type,
    sum(amount) AS montant_moyen_type_transaction
FROM df
GROUP BY type""").to_df()
rentabilité_transaction


In [ ]:
(df["isFlaggedFraud"] == 1).sum()

In [ ]:
(df["isFraud"]==1).sum()

In [ ]:
df.columns

In [ ]:
pourcentage_massive_fraude  = duckdb.sql("""
    SELECT nameOrig, amount , isFraud
    FROM df
    WHERE isFraud = 1 AND isFlaggedFraud = 1
""").to_df()

pourcentage_massive_fraude['isFraud'].sum()

In [ ]:
taux_de_fraud = duckdb.sql("""
    SELECT COUNT(*) AS total_transactions,
           SUM(CASE WHEN isFraud = 1 THEN 1 ELSE 0 END) AS total_fraudulent_transactions,
           (SUM(CASE WHEN isFraud = 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS fraud_rate
    FROM df
""").to_df()

taux_de_fraud


In [ ]:
taux_succe_transfer = duckdb.sql( """
    SELECT COUNT(*) AS total_transfers,
           SUM(CASE WHEN type = 'TRANSFER' THEN 1 ELSE 0 END) AS successful_transfers,
           (SUM(CASE WHEN type = 'TRANSFER' THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS success_transfer_rate
    FROM df
"""
).to_df()

taux_succe_transfer

In [ ]:
# Impact des transaction frauduleuse 
fraudulent_transactions_impact_query = duckdb.sql("""
    SELECT SUM(amount) AS total_amount_fraudulent_transactions
    FROM df
    WHERE isFraud = 1
""").to_df()

fraudulent_transactions_impact_query


##### Taux de Succès des Transfert

In [ ]:
taux_succé_transfer = duckdb.sql("""SELECT 
    SUM(CASE WHEN type = 'TRANSFER' THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS success_transfer_rate
FROM df""").to_df()

taux_succé_transfer


##### Taux de Signalement des Transactions Massives 

In [ ]:
taux_signalisation_transaction_massive =  duckdb.sql("""SELECT 
    round(SUM(CASE WHEN amount > 200000 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),2) AS flagged_transactions_rate
FROM df""").to_df()

taux_signalisation_transaction_massive


##### Ratio de Fraude par Type de Transaction 

In [ ]:
fraud_par_type_transaction= duckdb.sql("""SELECT 
    type,
    round(SUM(CASE WHEN isFraud = 1 THEN 1 ELSE 0 END) * 100/ COUNT(*),3) AS fraud_ratio_by_type
FROM df
GROUP BY type""").to_df()

fraud_par_type_transaction


In [ ]:
duckdb.sql("""SELECT 
    type,
    COUNT(*) AS nombre_transactions,
    round(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS pourcentage_total
FROM df
GROUP BY type
""").to_df()

In [ ]:
df.shape

In [ ]:
print('CASH_OUT', round(df['type'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('PAYMENT', round(df['type'].value_counts()[1]/len(df) * 100,2), '% of the dataset')
print('CASH_IN', round(df['type'].value_counts()[2]/len(df) * 100,2), '% of the dataset')
print('TRANSFER', round(df['type'].value_counts()[3]/len(df) * 100,2), '% of the dataset')
print('DEBIT', round(df['type'].value_counts()[4]/len(df) * 100,2), '% of the dataset')


#####   Fréquence des Transactions par Client :


In [ ]:
frequence_transaction_client = duckdb.sql("""SELECT 
    nameOrig,
    COUNT(*) AS transaction_count_per_client,
    (COUNT(*) * 1.0) / (SELECT COUNT(*) FROM df) AS frequency
FROM df
GROUP BY nameOrig
""").to_df()
frequence_transaction_client['transaction_count_per_client'].unique()

#### Évolution du Solde Moyen des Comptes Clients

In [ ]:
evolution_solde_moyen_compte_client = duckdb.sql("""SELECT 
    nameOrig,
    AVG(newbalanceOrig - oldbalanceOrg) AS average_balance_change
FROM df
GROUP BY nameOrig
""").to_df()

evolution_solde_moyen_compte_client.shape

In [ ]:
print(df['isFraud'].value_counts()[0]/len(df) * 100,2)
print(df['isFraud'].value_counts()[1]/len(df) * 100,2)

In [ ]:
df['type'].value_counts()

In [ ]:
var_catégorielle = ['type', 'nameOrig', 'nameDest']

colonnes_existantes = df.columns.tolist()
colonnes_a_supprimer = [colonne for colonne in var_catégorielle if colonne in colonnes_existantes]

df_num = df.drop(columns=colonnes_a_supprimer)
df_num

## Conservation des observations ou type = transfer ou cash_out 

In [ ]:
data1 = dt.query('type == "TRANSFER" or type == "CASH_OUT"')
data1['type'].unique()

In [ ]:
fraud_data = df[df['isFraud'] ==1]


In [ ]:
#spécifier l'axe des x et l'axe des y
ax = fraud_data['type'].value_counts().plot(
kind='pie', 
figsize=(8,4),
autopct = '%.2f%%', 
labels=fraud_data['type'].value_counts().index,
legend = True,
)

ax.set_title(
'Distributions des fraudes suivants les différents types', 
loc ='center',
fontsize=12, 
fontweight='bold'
)

plt.show()

In [ ]:
#spécifier l'axe des x et l'axe des y
ax = data['type'].value_counts().plot(
kind='pie', 
figsize=(8,4),
autopct = '%.2f%%', 
labels=df['type'].value_counts().index,
legend = True,
)

ax.set_title(
'Distribution des types de transaction', 
loc ='center',
fontsize=12, 
fontweight='bold'
)

plt.show()

In [ ]:
# Distribution des transactions frauduleuse suivant les différents type 

In [ ]:
var_catégorielle = ['nameOrig', 'nameDest']

colonnes_existantes = data1.columns.tolist()
colonnes_a_supprimer = [colonne for colonne in var_catégorielle if colonne in colonnes_existantes]

data = data1.drop(columns=colonnes_a_supprimer)
data.columns

In [ ]:
# Let's turn non-numerical values into numbers
for label, content in data.items():
    if not pd.api.types.is_numeric_dtype(content):
        data[label] = pd.Categorical(content).codes+1

data.shape


### Distribution  de fraude suivant type transfer ou cash_out 

In [ ]:
#spécifier l'axe des x et l'axe des y
ax = data['isFraud'].value_counts().plot(
kind='pie', 
figsize=(8,4),
autopct = '%.2f%%', 
labels=df['isFraud'].value_counts().index,
legend = True,
)

ax.set_title(
'Distributions des transactions fauduleuse et non frauduleuse', 
loc ='center',
fontsize=12, 
fontweight='bold'
)

plt.show()

En analysant les résultats obtenus à partir de la matrice de corrélation, on observe que 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest' présentent une corrélation très élevée (corrélation de 1). Afin d'éviter la multicollinéarité  (La multicollinéarité est un terme utilisé en statistiques et en analyse de régression pour décrire une situation où deux ou plusieurs variables indépendantes dans un modèle statistique sont fortement corrélées entre elles) dans le modèle, il est possible de supprimer l'une des variables, d'appliquer l'Analyse en Composantes Principales (ACP) ou encore d'étudier la contribution de chaque variable en fonction du modèle

In [ ]:
# Utilisons loption suprresion des l'un des variable

var_catégorielle = [ 'oldbalanceOrg', 'newbalanceDest']

colonnes_existantes = data.columns.tolist()
colonnes_a_supprimer = [colonne for colonne in var_catégorielle if colonne in colonnes_existantes]

data_model = data.drop(columns=colonnes_a_supprimer)
data_model .columns

## Text des modèles de Machine Learning 

In [ ]:
X = data_model.drop('isFraud', axis=1)
y = data_model['isFraud']

In [ ]:
X.columns

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Instanciation de PCA
acp = PCA()

# Calcul des composantes principales
coord = acp.fit_transform(X)

# Nombre de composantes calculées
print("Nombre de composantes:", acp.n_components_)

# Variance expliquée pour chaque composante
explained_variance = acp.explained_variance_ratio_

# Cumul de la variance expliquée
cumulative_explained_variance = np.cumsum(explained_variance)

# Affichage de la variance expliquée et du cumul
for i, (var, cum_var) in enumerate(zip(explained_variance, cumulative_explained_variance), 1):
    print(f"Composante {i}: Variance expliquée = {var:.4f}, Cumul de variance expliquée = {cum_var:.4f}")

# Visualisation du cumul de variance expliquée
import matplotlib.pyplot as plt

plt.plot(range(1, len(explained_variance) + 1), cumulative_explained_variance, marker='o')
plt.xlabel("Nombre de composantes principales")
plt.ylabel("Cumul de variance expliquée")
plt.title("Variance expliquée et cumul de variance expliquée")
plt.show()
threshold = 0.95  # Par exemple, choisir un seuil de 95%

num_components_to_retain = np.argmax(cumulative_explained_variance >= threshold) + 1

print(f"\nNombre de composantes pour conserver {threshold * 100}% de l'information: {num_components_to_retain}")


In [ ]:
np.random.seed(37)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Application de l'ACP pour réduire la dimensionnalité
n_components = 2 # Choix du nombre de composantes principales à conserver
pca = PCA(n_components=n_components)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

### Test avec un modèle de deep Learning 

### Random Forest 

In [ ]:
from sklearn.preprocessing import RobustScaler
sc=RobustScaler()
X_train=sc.fit_transform(X_train)
X_test= sc.fit_transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Définition du modèle RandomForestClassifier avec class_weight='balanced'
R1 = RandomForestClassifier(class_weight='balanced')

# Paramètres pour la Grid Search
param_grid_rf = {
    'n_estimators': [10, 80, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [10],
    'min_samples_split': [2, 3, 4]
}

# Appliquer la Grid Search pour RandomForestClassifier
grid_search_rf = GridSearchCV(R1, param_grid_rf, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Meilleurs paramètres trouvés
best_params_rf = grid_search_rf.best_params_
print("Best Parameters for RandomForestClassifier:", best_params_rf)

# Utiliser le modèle avec les meilleurs paramètres
R1_best = grid_search_rf.best_estimator_
print(R1_best)

# Entraînement du modèle sur les données d'entraînement
R1_best.fit(X_train, y_train)

# Prédiction et affichage de la matrice de confusion pour les données d'entraînement
pred_train = R1_best.predict(X_train)
c_train = confusion_matrix(y_train, pred_train)
print("Confusion Matrix (Train):")
print(c_train)

# Prédiction et affichage de la matrice de confusion pour les données de test
pred_test = R1_best.predict(X_test)
c_test = confusion_matrix(y_test, pred_test)
print("Confusion Matrix (Test):")
print(c_test)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train, pred_train)
test_accuracy = roc_auc_score(y_test, pred_test)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


In [ ]:
#Affichage
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")
print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')
plt.title('Matrice de confusion X_train')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.title('Matrice de confusion X_test')

plt.show()
print(classification_report(y_test, pred_test))
sns.heatmap

#### Test avec un model ensembliste : XGBClassifier 

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Define the XGBoost model
model = XGBClassifier()

# Define the parameter grid for the grid search
param_grid = {'n_estimators': [10, 80, 100],
              'gamma': [0, 0.01, 1],        
              'max_depth': [7, 8, 10], 
              'colsample_bytree': [0.7, 1],
              'learning_rate': [0.1, 0.01, 0.001],
              'scale_pos_weight': [99],  
             }

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(best_params)

# Train the model with the best parametersbe
model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

# Prediction and confusion matrices
predict_train = model.predict(X_train)
c_train = confusion_matrix(y_train, predict_train)

predict_test = model.predict(X_test)
c_test = confusion_matrix(y_test, predict_test)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train, predict_train)
test_accuracy = roc_auc_score(y_test, predict_test)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


In [ ]:
#Affichage
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")
print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')
plt.title('Matrice de confusion X_train')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.title('Matrice de confusion X_test')

plt.show()
print(classification_report(y_test, predict_test))
sns.heatmap

###  deep Learning Model 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])



In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')


In [ ]:

pred1 = (model.predict(X_train)> 0.5).astype("int32")
c_train = confusion_matrix(y_train, pred1)
pred = (model.predict(X_test)> 0.5).astype("int32")
c_test = confusion_matrix(y_test, pred)
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")


print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.show()
print(classification_report(y_test, pred))
sns.heatmap
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train, pred1)
test_accuracy = roc_auc_score(y_test, pred)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


# Sous-echantillonage
<a id="18"></a>


Dans cette partie nous essayons d'équilibrer les données de notre variable cible 'Class'. On utilisons Ainsi la méthode du sous-échantillonnage aléatoire ou 'Under-Sampling aléatoire L'objectif ici est de supprimer aléatoirement certaines observations de la classe majoritaire et de répéter l'opération jusqu'à ce que la classe majoritaire et minoritaire soit équilibrée.

Le sous-échantillonnage peut être un bon choix lorsque nos donnéessont de taille importante. Mais son inconvénient est que parfois nous supprimons les informations qui peuvent être précieuses.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sample=RandomUnderSampler()
X_train_samp, y_train_samp=sample.fit_resample(X_train,y_train )


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Définition du modèle RandomForestClassifier
R1 = RandomForestClassifier()

# Paramètres pour la Grid Search
param_grid_rf = {
    'n_estimators': [10, 80, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [10],
    'min_samples_split': [2, 3, 4]
}

# Appliquer la Grid Search pour RandomForestClassifier
grid_search_rf = GridSearchCV(R1, param_grid_rf, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search_rf.fit(X_train_samp, y_train_samp)

# Meilleurs paramètres trouvés
best_params_rf = grid_search_rf.best_params_
print("Best Parameters for RandomForestClassifier:", best_params_rf)

# Utiliser le modèle avec les meilleurs paramètres
R1_best = grid_search_rf.best_estimator_
print(R1_best)

# Entraînement du modèle sur les données d'entraînement
R1_best.fit(X_train_samp, y_train_samp)

# Prédiction et affichage de la matrice de confusion pour les données d'entraînement
pred_train = R1_best.predict(X_train)
c_train = confusion_matrix(y_train_samp, pred_train)
print("Confusion Matrix (Train):")
print(c_train)

# Prédiction et affichage de la matrice de confusion pour les données de test
pred_test = R1_best.predict(X_test)
c_test = confusion_matrix(y_test, pred_test)
print("Confusion Matrix (Test):")
print(c_test)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train_samp, pred1)
test_accuracy = roc_auc_score(y_test, pred)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


In [ ]:
#Affichage
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")
print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')
plt.title('Matrice de confusion X_train')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.title('Matrice de confusion X_test')

plt.show()
print(classification_report(y_test, pred))
sns.heatmap

#### XGBCLASSIER

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Define the XGBoost model
model = XGBClassifier()

# Define the parameter grid for the grid search
param_grid = {'n_estimators': [10, 80, 100],
              'gamma': [0, 0.01, 1],        
              'max_depth': [7, 8, 10], 
              'colsample_bytree': [0.7, 1],
              'learning_rate': [0.1, 0.01, 0.001],
              'scale_pos_weight': [99],  
             }

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1')
grid_search.fit(X_train_samp, y_train_samp)

# Get the best parameters
best_params = grid_search.best_params_

# Train the model with the best parameters
model = XGBClassifier(**best_params)
model.fit(X_train_samp, y_train_samp)

# Prediction and confusion matrices
predict_train = model.predict(X_train_samp)
c_train = confusion_matrix(y_train_samp, predict_train)

predict_test = model.predict(X_test)
c_test = confusion_matrix(y_test, predict_test)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train_samp, predict_train)
test_accuracy = roc_auc_score(y_test, predict_test)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


In [ ]:
#Affichage
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")
print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')
plt.title('Matrice de confusion X_train')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.title('Matrice de confusion X_test')

plt.show()
print(classification_report(y_test, predict_test))
sns.heatmap

### Deep Learning Model

In [ ]:

model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_samp, y_train_samp, epochs=10, batch_size=32, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

In [ ]:

pred1 = (model.predict(X_train_samp)> 0.5).astype("int32")
c_train = confusion_matrix(y_train_samp, pred1)
pred = (model.predict(X_test)> 0.5).astype("int32")
c_test = confusion_matrix(y_test, pred)
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")


print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.show()
print(classification_report(y_test, pred))
sns.heatmap
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train_samp, pred1)
test_accuracy = roc_auc_score(y_test, pred)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


## OverSampling 

In [ ]:
from imblearn.over_sampling import SMOTE
sample = SMOTE()
X_train_sampOver, y_train_sampOver = sample.fit_resample(X_train, y_train)
X_train_sampOver.shape, y_train_sampOver.shape

In [ ]:
from imblearn.over_sampling import RandomOverSampler
sample=RandomOverSampler()
X_train_sampOver, y_train_sampOver = sample.fit_resample(X_train, y_train)
X_train_sampOver.shape, y_train_sampOver.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Définition du modèle RandomForestClassifier
R1 = RandomForestClassifier()

# Paramètres pour la Grid Search
param_grid_rf = {
    'n_estimators': [10, 80, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [10],
    'min_samples_split': [2, 3, 4]
}

# Appliquer la Grid Search pour RandomForestClassifier
grid_search_rf = GridSearchCV(R1, param_grid_rf, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search_rf.fit(X_train_sampOver, y_train_sampOver)

# Meilleurs paramètres trouvés
best_params_rf = grid_search_rf.best_params_
print("Best Parameters for RandomForestClassifier:", best_params_rf)

# Utiliser le modèle avec les meilleurs paramètres
R1_best = grid_search_rf.best_estimator_
print(R1_best)

# Entraînement du modèle sur les données d'entraînement
R1_best.fit(X_train_sampOver, y_train_sampOver)

# Prédiction et affichage de la matrice de confusion pour les données d'entraînement
pred_train = R1_best.predict(X_train_sampOver)
c_train = confusion_matrix(y_train_sampOver, pred_train)
print("Confusion Matrix (Train):")
print(c_train)

# Prédiction et affichage de la matrice de confusion pour les données de test
pred_test = R1_best.predict(X_test)
c_test = confusion_matrix(y_test, pred_test)
print("Confusion Matrix (Test):")
print(c_test)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train_sampOver, pred_train)
test_accuracy = roc_auc_score(y_test, pred_test)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


In [ ]:
#Affichage
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")
print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')
plt.title('Matrice de confusion X_train')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.title('Matrice de confusion X_test')

plt.show()
print(classification_report(y_test, pred_test))
sns.heatmap

### XGBCLASSIER

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Define the XGBoost model
model = XGBClassifier()

# Define the parameter grid for the grid search
param_grid = {'n_estimators': [10, 80, 100],
              'gamma': [0, 0.01, 1],        
              'max_depth': [7, 8, 10], 
              'colsample_bytree': [0.7, 1],
              'learning_rate': [0.1, 0.01, 0.001],
              'scale_pos_weight': [99],  
             }

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1')
grid_search.fit(X_train_sampOver, y_train_sampOver)

# Get the best parameters
best_params = grid_search.best_params_
print(best_params)

# Train the model with the best parametersbe
model = XGBClassifier(**best_params)
model.fit(X_train_sampOver, y_train_sampOver)

# Prediction and confusion matrices
predict_train = model.predict(X_train_sampOver)
c_train = confusion_matrix(y_train_sampOver, predict_train)

predict_test = model.predict(X_test)
c_test = confusion_matrix(y_test, predict_test)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train_sampOver, predict_train)
test_accuracy = roc_auc_score(y_test, predict_test)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


In [ ]:
#Affichage
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")
print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')
plt.title('Matrice de confusion X_train')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.title('Matrice de confusion X_test')

plt.show()
print(classification_report(y_test, predict_test))
sns.heatmap

### Deep Learning Model

In [ ]:

model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_sampOver,y_train_sampOver, epochs=10, batch_size=32, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

In [ ]:

pred1 = (model.predict(X_train_sampOver)> 0.5).astype("int32")
c_train = confusion_matrix(y_train_sampOver, pred1)
pred = (model.predict(X_test)> 0.5).astype("int32")
c_test = confusion_matrix(y_test, pred)
print("Confusion Matrix")
print(pd.DataFrame(np.hstack([c_train, c_test]), index=["y=0", "y=1"],
                            columns="train:y=0 train:y=1 test:y=0 test:y=1".split()), "\n\n")


print(c_train)
plt.figure(figsize = (15,5))
plt.subplot(1,2,1)
sns.heatmap(c_train,annot=True,cmap='Blues',fmt='0')

plt.subplot(1,2,2)
sns.heatmap(c_test,annot=True,cmap='Reds',fmt='0',xticklabels='auto',
    yticklabels='auto')
plt.show()
print(classification_report(y_test, pred))
sns.heatmap
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
# evaluate
train_accuracy = roc_auc_score(y_train_sampOver, pred1)
test_accuracy = roc_auc_score(y_test, pred)
print("train accurency",train_accuracy)
print("test accurency",test_accuracy)


Précision (Precision) :

Pour la classe 0 (non frauduleuse) : 1.00 (100%)
Pour la classe 1 (frauduleuse) : 0.05 (5%)
La précision mesure le nombre de vrais positifs divisé par le nombre total de prédictions positives. Pour la classe 0, toutes les prédictions positives sont correctes, ce qui donne une précision de 100%. Cependant, pour la classe 1, seulement 5% des prédictions positives sont correctes.
Rappel (Recall ou Sensibilité) :

Pour la classe 0 (non frauduleuse) : 0.96 (96%)
Pour la classe 1 (frauduleuse) : 0.78 (78%)
Le rappel mesure le nombre de vrais positifs divisé par le nombre total d'instances réellement positives. Pour la classe 0, 96% des transactions non frauduleuses sont correctement identifiées. Pour la classe 1, 78% des transactions frauduleuses sont correctement identifiées.
Score F1 (F1 Score) :

Pour la classe 0 : 0.98
Pour la classe 1 : 0.09
Le score F1 est la moyenne harmonique de la précision et du rappel. Il prend en compte les faux positifs et faux négatifs. Une valeur élevée du score F1 indique un équilibre entre la précision et le rappel. La classe 0 a un score F1 élevé, mais la classe 1 a un score F1 relativement bas.
Accuracy (Précision globale) :

Accuracy : 0.95 (95%)
L'accuracy mesure le nombre total de prédictions correctes divisé par le nombre total d'instances. La précision globale est élevée à 95%, mais cela peut être trompeur dans des situations de classes très déséquilibrées, comme dans le cas des transactions frauduleuses.
Macro AVG (Moyenne Macro) :

Précision macro avg : 0.52
Rappel macro avg : 0.87
Score F1 macro avg : 0.53
La moyenne macro calcule les métriques pour chaque classe séparément et fait la moyenne. Elle donne à chaque classe un poids égal, indépendamment de sa taille.
Weighted AVG (Moyenne Pondérée) :

Précision weighted avg : 1.00
Rappel weighted avg : 0.95
Score F1 weighted avg : 0.97
La moyenne pondérée calcule les métriques pour chaque classe séparément, mais elle les pondère en fonction du nombre d'instances dans chaque classe. Cela donne une meilleure indication de la performance du modèle dans des situations de classes déséquilibrées.
En résumé, votre modèle semble avoir une très bonne précision globale, mais il présente des performances inégales entre les deux classes en raison du déséquilibre des données. La classe 1 (frauduleuse) a une précision et un rappel relativement bas, indiquant des difficultés à bien la prédire. Vous pourriez envisager d'explorer des techniques d'équilibrage de classe ou d'autres ajustements du modèle pour améliorer ces résultats.